In [ ]:
import timeit
# records the time at this instant
# of the program
runtime_start = timeit.default_timer()

# 1 Creating input files for pysumma

In this notebook, we set the study basins, and simulation period and put together forcing data setups for CAMELS.
This requires hourly forcings of
 - temperature (K)
 - precipitation (kg/m^2/s^1)
 - shortwave radiation (W/m^2)
 - longwave radiation (W/m^2)
 - specific humidity (g/g)
 - air pressure (Pa)
 - wind speed (m/s)
 
NLDAS hourly forcings are considered to be "truth". 
We take NLDAS hourly forcings and give them their mean daily values for each variable in turn (or all variables). 
Then, we take NLDAS daily forcings of maximum and minimum temperature, total precipitation, and mean windspeed; and redistribute them to hourly with calculation of the other variables using the MetSim python package. 

<br>

## 1_1 Preliminary steps
### 1_1_1 Check the enviroment
Check that we loaded correct environment. This should show pysumma version 3.0.3.

In [ ]:
conda list summa

<br>

### 1_1_2 Import libraries
Import the required librarires.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import geoviews as gv
import geopandas as gpd
import holoviews as hv
import xarray as xr
import ogr
import cartopy
import os
import ipyplot

hv.notebook_extension('bokeh')


## 1_2 Set up paths to SUMMA configuration files for CAMELS basins
### 1_2_1 Unzip the folder contatining SUMMA CAMELS configurations

You will need to unzip `summa_camels_hydroshare.zip` data if it is not already unzipped. This folder contains the SUMMA configuration, local attributes, and parameters files that were created using CAMELS.

In [ ]:
# TODO: To be more organized, unzip in a folder inside the contents instead of unzipping at the content level with no folder!
if not os.path.exists("summa_camels"):
    !unzip summa_camels_hydroshare.zip
else:
    pass

<br>

### 1_2_2 Set up paths to SUMMA configuration files
Set up the paths to summa_camels_hydroshare subfolders for the simulations in this notebook and next ones.

In [ ]:
top_folder = os.path.join(os.getcwd(), 'summa_camels')
settings_folder = os.path.join(top_folder, 'settings')
# Shapefile containing the 671 CAMELS basins with their attributes (e.g., hru_id)
shapefile = os.path.join(os.getcwd(), 'basin_set_full_res_simple/HCDN_nhru_final_671.shp')

<br>

## 1_3 Select basins and simlaution period

We will use a setup which uses CAMELS basin as lumped, so considered as one "hydrologic response unit" or HRU.
A HRU typically delineates a watershed by topography, soil type, land use, or other defining feature.
There are a possible 671 CAMELS basins to use; below you can choose the ID. Because this is an unstructured mesh we will be defining the mesh elements by their respective basin identifiers.

If you haven't done already, download NLDAS forcing data from HydroShare OpenDAP (http://hyrax.hydroshare.org/opendap/hyrax/)

### 1_3_1 Retrieve the meteorological forcings

In [ ]:
# Using Hydroshare THREDDS Service retrieve the nldas Forcing (1980 to 2018) from the first HydroShare resource: a28685d2dd584fe5885fc368cb76ff2
extra_vars0 = xr.open_dataset('http://thredds.hydroshare.org/thredds/dodsC/hydroshare/resources/a28685d2dd584fe5885fc368cb76ff2a/data/contents/nldasForcing1980to2018.nc')
# TODO: confrim what the next line does: set time step as hourly? (3600 s =1 hour?!) also, it is already hourly why do we use 3600?
extra_vars0['data_step'] = 3600

### 1_3_2 Select basins and simulation period

<div class="alert alert-block alert-danger">
<b> Select simulation period and basins:  </b> You will select a time period out of this to run the simulations. You cannot run fewer basins though, so choose wisely as every basin added means more computational expense.   </div> 

In [ ]:
# Select start and end dates for simulations
start_date = '1990-09-30'
end_date = '1996-09-30'

# All HRUs, select out all or some to look at
#the_hru = np.array(extra_vars0['hruId']) #run all
hru_ids = [1632900] #HRU_ID: 1632900 is pre-populated, other ones from the paper are 2212600, 9378630, 11264500
# hru_ids = [1632900, 2212600, 9378630, 11264500]


### 1_3_3 Slice the forcings to selected basins and simulaiotn period

In [ ]:
the_hru = np.array(hru_ids)

# In this study, we assume we have GRUs (grouped response units) same as HRUs while GRUs can be made up of one or more HRU
the_gru = the_hru

# Slice to seleced HRUs
extra_vars1 = extra_vars0.sel(hru=the_hru)

# Slice to selected time period
extra_vars = extra_vars1.loc[dict(time=slice(start_date, end_date))] 
print(the_hru)

### 1_3_4 Slice the SUMMA CAMELS shapefile to selected basins

In [ ]:
gdf = gpd.read_file(shapefile)
shapes = cartopy.io.shapereader.Reader(shapefile)
list(shapes.records())[gdf[gdf['hru_id']==hru_ids[0]].index.values[0]] #just to see what an entry looks like

In [ ]:
# Convert the data from an xarray dataset to a pandas dataframe 
out_df = extra_vars0['hru']
out_df = out_df.to_dataframe()
# Make sure we have some metadata to join with the shapefile
out_df['hru_id'] = gdf['hru_id'].values
#search for the ones with desired records
find_rec = out_df.loc[the_hru,:]['hru_id']
#look at attributes
desired_shapes = []
for i in find_rec:
    for s in shapes.records():
        if s.attributes['hru_id'] == i :
            desired_shapes.append(s)

### 1_3_5 Show the selected basins in map
Once the next cell is ran successfully, use the tools on top right of the map to interact with the map. 

Basins are colored by index values and are hoverable for index values and IDs. Selected basins are higlighted in cyan. You can look up other basin IDs on this interactive map if you'd like to explore them instead (changing the selection above). 

In [ ]:
# Create backgound
mapp =gv.tile_sources.StamenTerrainRetina.opts(width=900, height=500)
# Create the shape plot
poly = gv.Shape.from_records(shapes.records(), out_df, index=['hru_id'], on='hru_id',crs=cartopy.crs.PlateCarree())
poly2 = gv.Shape.from_records(desired_shapes,out_df.loc[the_hru,:],on='hru_id', crs=cartopy.crs.PlateCarree())
poly = poly.opts(cmap='plasma', tools=['hover'], colorbar=True, alpha=0.8)
poly2 = poly2.opts(fill_color='cyan', line_color='cyan', alpha=0.8)
# Plot
mapp*poly*poly2

If you are using 10 or fewer basins, the cell below will plot each basin in detail. Multiple HRUs per basin will be shown in red; we are not using that distributed setup but instead combining the HRUs into a single lumped HRU basin.

In [ ]:
if len(the_hru)<=10:
    for i in the_hru:
        if len(str(i)) <=7:
            images_array1 = ['https://staff.ral.ucar.edu/wood/watersheds/dem_figs/0'+str(i)+'.dem.png', 'https://staff.ral.ucar.edu/wood/watersheds/basin_figs/0'+str(i)+'.watershed.png']
        else:
            images_array1 = ['https://staff.ral.ucar.edu/wood/watersheds/dem_figs/'+str(i)+'.dem.png', 'https://staff.ral.ucar.edu/wood/watersheds/basin_figs/'+str(i)+'.watershed.png']
        ipyplot.plot_images(images_array1, max_images=20, img_width=400)
the_hru

<br>

### 1_3_6 Slice the SUMMA CAMELS paramters and attributes files to the selected basins 
We have to select out only the HRU IDs of the basins we are using.

In [ ]:
# Attributes 
attrib = xr.open_dataset(settings_folder+'/attributes.camels.v2.nc')
attrib = attrib.assign_coords(hru=attrib['hruId'])
attrib = attrib.assign_coords(gru=attrib['gruId'])
gg = attrib['gruId'] # save because gruId was missing from the parameter file
attrib = attrib.sel(hru=the_hru)
attrib = attrib.sel(gru=the_gru)
attrib = attrib.drop(['hru','gru']) #summa doesn't like these to have coordinates
attrib.to_netcdf(settings_folder+'/attributes.nc')

In [ ]:
# Parameters
param = xr.open_dataset(settings_folder+'/trialParams.camels.Oct2020.nc')
param = param.assign_coords(hru=param['hruId'])
param = param.assign_coords(gru=gg) # there should be a gruId in here, but there wasn't
param = param.sel(hru=the_hru)
param = param.sel(gru=the_gru)
param = param.drop(['hru','gru']) #summa doesn't like these to have coordinates
param.to_netcdf(settings_folder+'/parameters.nc')

<br>

### 1_3_7 Make the constant initial conditions
Lastly, we will need to make the constant initial conditions file, for 8 constant soil layers and 0 snow layers at the start of the simulation.

In [ ]:
!cd {settings_folder}; source activate /opt/conda/envs/pysumma; python gen_coldstate.py attributes.nc init_cond.nc int

<br>

## 1_4 Create forcing files with constant daily values at their daily means
Here we make the NLDAS data hourly values into constant 24 hourly values for each forcing variable. 
We need to make these 24 hours represent a local day, so local time zones, such that later calculations on days work. We will shift the days so that the time zones line up, make constant, and shift back. As a schematic:

![](constantDay.jpg)

<br>

### 1_4_1 Write and save the truth forcing
Write this file for pysumma forcing and save the forcing file name.

In [ ]:
%%time
truth = extra_vars
t0 = truth['time'].values[0] 
tl = truth['time'].values[-1]
t0_s = pd.to_datetime(str(t0))
t0_sf =t0_s.strftime('%Y%m%d')
tl_s = pd.to_datetime(str(tl))
tl_sf =tl_s.strftime('%Y%m%d')
ffname ='NLDAStruth_' + t0_sf +'-' + tl_sf +'.nc'
truth.to_netcdf(top_folder+'/data/truth/'+ffname)
fflistname = settings_folder+'/forcingFileList.truth.txt' 
file =open(fflistname,"w")
file.write(ffname)
file.close()

### 1_4_2 Shifting to local time zones using longitude values

<br>
To make the daily data, first need to shift to local time zones with longitude.

In [ ]:
# Separate out variables with no time dimension and add offset
ds = truth
ds_withtime = ds.drop_vars([ var for var in ds.variables if not 'time' in ds[var].dims ])
ds_timeless = ds.drop_vars([ var for var in ds.variables if     'time' in ds[var].dims ])
DEG_PER_REV = 360.0       # Number of degrees in full revolution
HRS_PER_DAY = 24
offset = (attrib['longitude'] / DEG_PER_REV) * HRS_PER_DAY
offset = offset.astype(int)
ds_withtime['offset'] = offset
ds_withtime = ds_withtime.assign_coords(hru=ds_timeless['hruId'])

<br>
In the next cell, the time zone changes. 
This takes about a minute using all 671 basins; a subset of basins should be shorter

In [ ]:
%%time
for t in np.unique(offset.data):
    ds = ds_withtime.where(offset==t,drop=True)
    ds = ds.shift(time=t)
    if t==np.unique(offset.data)[0]: ds0 = ds
    if t>np.unique(offset.data)[0]: ds0 = xr.concat([ds0,ds],dim='hru')
    print(t)

In [ ]:
# Sort back, and drop offset
ds_withtime = ds0.sortby('hru')
ds_withtime = ds_withtime.drop_vars('offset') 

<br>

### 1_4_3 Downsample hourly time-series data to daily data
Downsample hourly time-series data to daily data. 
This takes about a 30 seconds.

In [ ]:
%%time
truth24 = xr.merge([ds_timeless, ds_withtime.resample(time='1D').mean()]).load()
# Fix time encoding to be the same since the merge drops it
truth24.time.encoding = extra_vars.time.encoding

<br>

### 1_4_4 Upsample back to hourly data and undo time zone changes
Then we upsample this back to hourly data for constant daily values. 
We need to undo the time zone changes after we upsample. 
This whole process takes about a minute using all 671 basins; a subset of basins should be shorter.

In [ ]:
# Add a fake day of data so upsamples until the end
day_fake = truth24.isel(time=-1)['time']+np.timedelta64(1,'D')
add_fake = truth24.isel(time=-1)
add_fake['time'] = day_fake
truth24_add = xr.concat([truth24, add_fake], dim='time',data_vars='minimal')

In [ ]:
%%time
# Again we have to separate out variables with no time dimension.
ds = truth24_add
ds_withtime = ds.drop_vars([ var for var in ds.variables if not 'time' in ds[var].dims ])
ds_timeless = ds.drop_vars([ var for var in ds.variables if     'time' in ds[var].dims ])
ds_withtime = ds_withtime.resample(time='1H').ffill()
ds_withtime['offset'] = offset
ds_withtime = ds_withtime.assign_coords(hru=ds_timeless['hruId'])
for t in np.unique(offset.data):
    ds = ds_withtime.where(offset==t,drop=True)
    ds = ds.shift(time=-t)
    if t==np.unique(offset.data)[0]: ds0 = ds
    if t>np.unique(offset.data)[0]: ds0 = xr.concat([ds0,ds],dim='hru')
    print(t)

In [ ]:
# Sort back, and drop offset, and merge
ds_withtime = ds0.sortby('hru')
ds_withtime = ds_withtime.drop_vars('offset') 
constant_all = xr.merge([ds_timeless, ds_withtime])
constant_all = constant_all.sel(hru=the_hru) #put back in original order

In [ ]:
# Take extra day off
constant_all = constant_all.isel(time=slice(0,-1))
# Fix time encoding to be the same since the merge drops it
constant_all.time.encoding = extra_vars.time.encoding

<br>

### 1_4_5 Scale constant SW radiation
We must correct shortwave radiation since SUMMA will impose that it is 0 when the sun is below the horizon. So, we distribute the constant shortwave radiation only during the daylight hours, using the hours NLDAS is 0 as night. Then we increase the magnitude of the shortwave radiation so that the dailys sum of the constant value with the 0 at night is the original. As a schematic:

![](constantSWR.jpg)

Other changes inside SUMMA are that specific humidity will be lowered in order that relative humidity does not exceed 100%, and tiny windspeeds will be eliminated.

In [ ]:
# Find where 0's shoud be based on original NLDAS data
zero_one = truth['SWRadAtm']/truth['SWRadAtm']
zero_one = zero_one.fillna(0)

In [ ]:
# Find how much too small shortwave is each day if we use these 0's
swr0 = zero_one*constant_all['SWRadAtm']
div = swr0.resample(time='1D').mean()/constant_all['SWRadAtm'].resample(time='1D').mean()

In [ ]:
# Upsample, again add a fake day of data so upsamples until the end
add_fake = div.isel(time=-1)
add_fake['time'] = day_fake
div_add = xr.concat([div, add_fake], dim='time')
div_add = div_add.resample(time='1H').ffill()

# Take extra day off
div = div_add.isel(time=slice(0,-1))

In [ ]:
# Finally add back in this constant shortwave radiation
swr0 = swr0/div
constant_all['SWRadAtm']=swr0

<br>

### 1_4_6 Create files with only one variable constant at their daily values
Now make files with only one variable held at daily means and save forcing file names.

In [ ]:
t0 = constant_all['time'].values[0] 
tl = constant_all['time'].values[-1]
t0_s = pd.to_datetime(str(t0))
t0_sf =t0_s.strftime('%Y%m%d')
tl_s = pd.to_datetime(str(tl))
tl_sf =tl_s.strftime('%Y%m%d')

In [ ]:
%%time
constant_vars=['airpres','airtemp','LWRadAtm','pptrate','spechum','SWRadAtm','windspd']
for v in constant_vars:
    constant_one = truth.copy()
    constant_one[v]= constant_all[v]
    ffname ='NLDASconstant_' + v +'_forcing_' + t0_sf +'-' + tl_sf +'.nc'
    constant_one.to_netcdf(top_folder+'/data/constant/'+ffname)
    fflistname = settings_folder+'/forcingFileList.constant_' + v + '.txt' 
    file =open(fflistname,"w")
    file.write(ffname)
    file.close()
    print(v)

<br>

## 1_5 Check processed forcing files through plotting

To make sure things look how we want, we plot the constant dataset against the NLDAS "truth" dataset, and plot the cumulative variables to see how errors are compounding. 
We plot one HRU (the first one) for 2 months.

### 1_5_1 Hourly plots of the forcings

In [ ]:
#Plot hourly
fig, axes = plt.subplots(nrows=7, ncols=1, figsize=(20, 20))
axes = axes.flatten()
axes[0].set_title('Hourly')

unit_str = ['($ ^o K$)', '($kg/m/s$)', '($W/m^2$)','($w/m^2$)','($g/g$)','($Pa$)', '($m/s$)',]

#start =  24*9*30 #summer
start =  24*15*30 #winter
stop = start + 2*30*24 
truth_plt = truth.isel(hru=0, time=slice(start, stop))
constant_all_plt = constant_all.isel(hru=0, time=slice(start, stop))

for idx, var in enumerate(constant_vars):
    truth_plt[var].plot(ax=axes[idx],label='NLDAS')
    constant_all_plt[var].plot(ax=axes[idx],label='Constant')
    axes[idx].set_title('') 
    axes[idx].set_ylabel('{} {}'.format(var, unit_str[idx]), fontsize=18)
    axes[idx].set_xlabel('Date',fontsize=18)
    axes[idx].tick_params(axis='both', which='major', labelsize=13)
plt.tight_layout()
plt.legend()

### 1_5_2 Cummulative plots of the forcings

In [ ]:
#Plot cummulative
fig, axes = plt.subplots(nrows=7, ncols=1, figsize=(20, 20))
axes = axes.flatten()
axes[0].set_title('Cumulative')

truth_plt = truth.isel(hru=0, time=slice(start, stop)).cumsum(dim='time')
constant_all_plt = constant_all.isel(hru=0, time=slice(start, stop)).cumsum(dim='time')

for idx, var in enumerate(constant_vars):
    truth_plt[var].plot(ax=axes[idx],label='NLDAS')
    constant_all_plt[var].plot(ax=axes[idx],label='Constant')
    axes[idx].set_title('') 
    axes[idx].set_ylabel('{} {}'.format(var, unit_str[idx]), fontsize=18)
    axes[idx].set_xlabel('Date', fontsize=18)
    axes[idx].tick_params(axis='both', which='major', labelsize=13)
plt.tight_layout()
plt.legend()

In [ ]:
# Uncomment the following lines, if you need to clean up your directory. Note that if you do so, the rest of notebooks will not run as notebook 1 outputs are inputs for them.
# To avoid denied permission when deleting files, as some files are used by other notebooks, first shutdown the kernels for all notebooks, then set the kernel for this notebook,
# run, this block, and once the folders are removed, comment this block again.

# import os, shutil
# CURRENT_PATH = os.getcwd()
# CURRENT_PATH

# path = '/home/jovyan/work/Downloads/50e9716922dc487981b71e2e11f3bb5d/50e9716922dc487981b71e2e11f3bb5d/data/contents/dask-worker-space'
# !rm -rf {path}

# path = '/home/jovyan/work/Downloads/50e9716922dc487981b71e2e11f3bb5d/50e9716922dc487981b71e2e11f3bb5d/data/contents/summa_camels'
# !rm -rf {path}

# path = '/home/jovyan/work/Downloads/50e9716922dc487981b71e2e11f3bb5d/50e9716922dc487981b71e2e11f3bb5d/data/contents/regress_data'
# !rm -rf {path}

# path = '/home/jovyan/work/Downloads/50e9716922dc487981b71e2e11f3bb5d/50e9716922dc487981b71e2e11f3bb5d/data/contents/basin_set_full_res_simple'
# !rm -rf {path}

In [ ]:
# Print the time spent running the entire notebook.
# records the time at this instant 
# of the program
runtime_end = timeit.default_timer()

# printing the execution time by subtracting
# the time before the function from
# the time after the function
print(int(runtime_end-runtime_start), ' seconds')